# Table of Contents
* [Libraries](#Libraries)
* [Example Requests](#Example-Requests)
	* &nbsp;
		* [Header Diagnosis](#Header-Diagnosis)
* [Accessing Play by Play Data via API](#Accessing-Play-by-Play-Data-via-API)
* [Quarterly Game Data by Team](#Quarterly-Game-Data-by-Team)
	* [With no iterations](#With-no-iterations)
		* [Grab desired arguments](#Grab-desired-arguments)
		* [Load Arguments](#Load-Arguments)
		* [Grab data](#Grab-data)
	* [Prep for Quarterly Data with Iterations](#Prep-for-Quarterly-Data-with-Iterations)
* [Data Acquisition Script](#Data-Acquisition-Script)
* [Functions and Constants](#Functions-and-Constants)


# Libraries


In [1]:
import requests
import pandas as pd
import numpy as np

import cPickle as pickle

import urllib2


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# Example Requests


* Load the page
* Check network, XHR filters
* Grab the relevant request url

For example: This [initial link](http://stats.nba.com/league/player/#!/advanced/?sort=PIE&dir=1&Season=2014-15&SeasonType=Pre%20Season) leads to [target link](http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2014-15&SeasonSegment=&SeasonType=Pre+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=)

In [2]:
# Example

url="http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2014-15&SeasonSegment=&SeasonType=Pre+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="

response=requests.get(url)
response.raise_for_status()
#data = response.json()['resultSets'][0]['rowSet']

### Header Diagnosis

In [228]:
# Checkout keys to find appropriate key
response.json().keys()

[u'resource', u'resultSets', u'parameters']

In [7]:
# Convert headers and stats to frame



In [29]:
    
    
# View the headers
headers = response.json()['resultSets'][0]['headers']

# Looks like a list of 512 stats
stats = response.json()['resultSets'][0]['rowSet']

df = pd.DataFrame(stats,columns = headers )

df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,USG_PCT,PACE,PIE,FGM,FGA,FGM_PG,FGA_PG,FG_PCT,CFID,CFPARAMS
0,201985,A.J. Price,1610612756,PHX,28,6,4,2,0.667,13.3,...,0.217,95.68,0.106,15,33,2.5,5.5,0.455,5,"201985,1610612756"
1,201166,Aaron Brooks,1610612741,CHI,30,8,4,4,0.500,19.5,...,0.230,94.30,0.072,22,59,2.8,7.4,0.373,5,"201166,1610612741"
2,203905,Aaron Craft,1610612744,GSW,24,4,2,2,0.500,14.7,...,0.130,98.15,0.081,5,10,1.3,2.5,0.500,5,"203905,1610612744"
3,203932,Aaron Gordon,1610612753,ORL,19,8,4,4,0.500,22.2,...,0.183,100.18,0.097,26,56,3.3,7.0,0.464,5,"203932,1610612753"
4,203519,Adonis Thomas,1610612754,IND,22,4,2,2,0.500,8.9,...,0.142,94.97,0.025,2,10,0.5,2.5,0.200,5,"203519,1610612754"


# Accessing Play by Play Data via API

Example URL:

`http://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=0021500020&RangeType=2&Season=2015-16&SeasonType=Regular+Season&StartPeriod=1&StartRange=0`


In [3]:
playurl = "http://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=0021500020&RangeType=2&Season=2015-16&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"


In [4]:
response=requests.get(playurl)
response.raise_for_status()


In [7]:
df = response_to_frame(response)

In [8]:
df.head(2)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,0021500020,0,12,0,1,10:51 PM,12:00,None,None,None,...,None,None,None,0,0,None,NaN,None,None,None
1,0021500020,1,10,0,1,10:51 PM,12:00,Jump Ball Jordan vs. Pachulia: Tip to Griffin,None,None,...,Dallas,Mavericks,DAL,4,201933,Blake Griffin,1610612746,Los Angeles,Clippers,LAC


In [10]:
cols=['EVENTNUM','EVENTMSGACTIONTYPE','PERIOD','HOMEDESCRIPTION','NEUTRALDESCRIPTION','VISITORDESCRIPTION','SCORE','SCOREMARGIN']

In [19]:
# Score margin for target for modeling
df[~df['SCOREMARGIN'].isnull()][cols]

,EVENTNUM,EVENTMSGACTIONTYPE,PERIOD,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
2,2,1,1,Redick 20' Jump Shot (2 PTS) (Stephenson 1 AST),None,None,0 - 2,2
11,11,1,1,None,None,Jenkins 21' Jump Shot (2 PTS) (Pachulia 1 AST),2 - 2,TIE
14,14,78,1,None,None,Felton 12' Floating Jump Shot (2 PTS),4 - 2,-2
16,16,11,1,Griffin Free Throw 1 of 2 (1 PTS),None,None,4 - 3,-1
19,21,1,1,None,None,Felton 20' Jump Shot (4 PTS),6 - 3,-3
29,31,52,1,Jordan 1' Alley Oop Dunk (2 PTS) (Griffin 1 AST),None,None,6 - 5,-1
33,35,108,1,Griffin 3' Cutting Dunk Shot (3 PTS) (Paul 1 AST),None,None,6 - 7,1
40,42,11,1,Paul Free Throw 1 of 2 (1 PTS),None,None,6 - 8,2
41,44,12,1,Paul Free Throw 2 of 2 (2 PTS),None,None,6 - 9,3
42,47,47,1,None,None,Powell 15' Turnaround Jump Shot (2 PTS) (Felto...,8 - 9,1


In [20]:
# Score margin for target for modeling
df[df['SCOREMARGIN'].isnull()][cols]

,EVENTNUM,EVENTMSGACTIONTYPE,PERIOD,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0,0,1,None,None,None,None,None
1,1,0,1,Jump Ball Jordan vs. Pachulia: Tip to Griffin,None,None,None,None
3,3,5,1,Jordan BLOCK (1 BLK),None,MISS Felton 8' Layup,None,None
4,4,0,1,None,None,Evans REBOUND (Off:1 Def:0),None,None
5,5,1,1,None,None,MISS Pachulia 19' Jump Shot,None,None
6,6,0,1,Jordan REBOUND (Off:0 Def:1),None,None,None,None
7,7,1,1,MISS Griffin 19' Jump Shot,None,None,None,None
8,8,0,1,None,None,Nowitzki REBOUND (Off:0 Def:1),None,None
9,9,1,1,None,None,MISS Nowitzki 25' 3PT Jump Shot,None,None
10,10,0,1,None,None,Pachulia REBOUND (Off:1 Def:0),None,None


In [9]:
df.head(1).T

,0
GAME_ID,0021500020
EVENTNUM,0
EVENTMSGTYPE,12
EVENTMSGACTIONTYPE,0
PERIOD,1
WCTIMESTRING,10:51 PM
PCTIMESTRING,12:00
HOMEDESCRIPTION,None
NEUTRALDESCRIPTION,None
VISITORDESCRIPTION,None


## Data on Per-Play Basis Excludes TO and Rebounds as Plays

In [48]:
# Determine number of plays
# Wondering if 4th quarter LAC plays = 28
print "Total Plays"
for P in [1,2,3,4]:
    print "Plays in Period %s = %s" % (str(P),str(len(df[df.PERIOD ==P])))

Total Plays
Plays in Period 1 = 134
Plays in Period 2 = 136
Plays in Period 3 = 124
Plays in Period 4 = 108


In [49]:
print "LAC Plays"
for P in [1,2,3,4]:
    print "Plays in Period %s = %s" % (str(P),str(len(df[~df.HOMEDESCRIPTION.isnull()][df.PERIOD ==P])))

LAC Plays
Plays in Period 1 = 74
Plays in Period 2 = 69
Plays in Period 3 = 56
Plays in Period 4 = 57


/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:1819: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [68]:
fourth = df[~df.HOMEDESCRIPTION.isnull()][df.PERIOD ==4]

In [79]:
fourth

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
395,0021500020,454,2,45,4,12:50 AM,11:45,MISS Pierce 25' 3PT Jump Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
397,0021500020,456,2,42,4,12:51 AM,11:24,W. Johnson BLOCK (1 BLK),None,MISS Nowitzki 4' Driving Layup,...,None,None,None,4,202325,Wesley Johnson,1610612746,Los Angeles,Clippers,LAC
398,0021500020,457,4,0,4,12:51 AM,11:22,Smith REBOUND (Off:1 Def:2),None,None,...,None,None,None,0,0,None,NaN,None,None,None
399,0021500020,458,1,98,4,12:51 AM,11:00,Rivers 2' Cutting Layup Shot (9 PTS) (Pierce 2...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
400,0021500020,459,6,1,4,12:51 AM,10:55,Smith P.FOUL (P4.T1) (S.Corbin),None,None,...,Dallas,Mavericks,DAL,1,0,None,NaN,None,None,None
402,0021500020,461,6,2,4,12:52 AM,10:43,Pierce S.FOUL (P2.T2) (E.Dalen),None,None,...,Dallas,Mavericks,DAL,1,0,None,NaN,None,None,None
404,0021500020,464,1,1,4,12:52 AM,10:30,Rivers 26' 3PT Jump Shot (12 PTS) (Crawford 1 ...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
406,0021500020,466,4,0,4,12:53 AM,10:14,Pierce REBOUND (Off:0 Def:3),None,None,...,None,None,None,0,0,None,NaN,None,None,None
407,0021500020,467,1,43,4,12:53 AM,10:08,W. Johnson 4' Alley Oop Layup (4 PTS) (Crawfor...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
409,0021500020,469,4,0,4,12:53 AM,9:47,Smith REBOUND (Off:1 Def:3),None,None,...,None,None,None,0,0,None,NaN,None,None,None


In [80]:
print "Number of 4th quarter...\nfouls: %s" % fourth.HOMEDESCRIPTION.str.contains("FOUL").sum()
print "Free throws: %s" % fourth.HOMEDESCRIPTION.str.contains("Free").sum()
print "Rebounds: %s" % fourth.HOMEDESCRIPTION.str.contains("Rebound").sum()

Number of 4th quarter...
fouls: 3
Free throws: 4
Rebounds: 2


In [100]:
# Maybe exclude when EVENTMSGACTIONTYPE is 0 (rebound) or 1 (TO)
fourth[(fourth["EVENTMSGACTIONTYPE"] != 0) & (fourth["EVENTMSGACTIONTYPE"] != 1) ]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
395,0021500020,454,2,45,4,12:50 AM,11:45,MISS Pierce 25' 3PT Jump Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
397,0021500020,456,2,42,4,12:51 AM,11:24,W. Johnson BLOCK (1 BLK),None,MISS Nowitzki 4' Driving Layup,...,None,None,None,4,202325,Wesley Johnson,1610612746,Los Angeles,Clippers,LAC
399,0021500020,458,1,98,4,12:51 AM,11:00,Rivers 2' Cutting Layup Shot (9 PTS) (Pierce 2...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
402,0021500020,461,6,2,4,12:52 AM,10:43,Pierce S.FOUL (P2.T2) (E.Dalen),None,None,...,Dallas,Mavericks,DAL,1,0,None,NaN,None,None,None
407,0021500020,467,1,43,4,12:53 AM,10:08,W. Johnson 4' Alley Oop Layup (4 PTS) (Crawfor...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
411,0021500020,471,3,11,4,12:53 AM,9:39,MISS Smith Free Throw 1 of 2,None,None,...,None,None,None,0,0,None,NaN,None,None,None
415,0021500020,476,3,12,4,12:54 AM,9:39,MISS Smith Free Throw 2 of 2,None,None,...,None,None,None,0,0,None,NaN,None,None,None
419,0021500020,481,2,3,4,12:55 AM,9:12,MISS Smith 5' Hook Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
421,0021500020,483,2,97,4,12:55 AM,9:12,MISS Smith 4' Tip Layup Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
423,0021500020,485,1,97,4,12:55 AM,9:12,Smith 1' Tip Layup Shot (5 PTS),None,None,...,None,None,None,0,0,None,NaN,None,None,None


In [102]:
fourth[(fourth["EVENTMSGACTIONTYPE"] != 0) & (fourth["EVENTMSGACTIONTYPE"] != 1) ]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
395,0021500020,454,2,45,4,12:50 AM,11:45,MISS Pierce 25' 3PT Jump Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
397,0021500020,456,2,42,4,12:51 AM,11:24,W. Johnson BLOCK (1 BLK),None,MISS Nowitzki 4' Driving Layup,...,None,None,None,4,202325,Wesley Johnson,1610612746,Los Angeles,Clippers,LAC
399,0021500020,458,1,98,4,12:51 AM,11:00,Rivers 2' Cutting Layup Shot (9 PTS) (Pierce 2...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
402,0021500020,461,6,2,4,12:52 AM,10:43,Pierce S.FOUL (P2.T2) (E.Dalen),None,None,...,Dallas,Mavericks,DAL,1,0,None,NaN,None,None,None
407,0021500020,467,1,43,4,12:53 AM,10:08,W. Johnson 4' Alley Oop Layup (4 PTS) (Crawfor...,None,None,...,Los Angeles,Clippers,LAC,0,0,None,NaN,None,None,None
411,0021500020,471,3,11,4,12:53 AM,9:39,MISS Smith Free Throw 1 of 2,None,None,...,None,None,None,0,0,None,NaN,None,None,None
415,0021500020,476,3,12,4,12:54 AM,9:39,MISS Smith Free Throw 2 of 2,None,None,...,None,None,None,0,0,None,NaN,None,None,None
419,0021500020,481,2,3,4,12:55 AM,9:12,MISS Smith 5' Hook Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
421,0021500020,483,2,97,4,12:55 AM,9:12,MISS Smith 4' Tip Layup Shot,None,None,...,None,None,None,0,0,None,NaN,None,None,None
423,0021500020,485,1,97,4,12:55 AM,9:12,Smith 1' Tip Layup Shot (5 PTS),None,None,...,None,None,None,0,0,None,NaN,None,None,None


#  Quarterly Game Data by Team

## With no iterations

### Grab desired arguments

In [16]:
# Get gamedates as list
gamedates = get2014gamedates("Clippers","Warriors")

### Load Arguments

In [17]:
#  Initialize payload
payload = init_payload()

In [18]:
newargs = {
    'DateFrom':"1-1-1900",
    'DateTo':"1-1-1900",
    'MeasureType':'Base',
    'Period':'1',
    'TeamID':'1610612746',
    'OpponentTeamID':'1610612744'
    
}

### Grab data

## Prep for Quarterly Data with Iterations

In [13]:
# Set additional iterations needed
Periods = ["1","2","3","4"]
MeasureTypes = ["Base","Advanced","Misc","Four Factors","Scoring","Opponent"] # "Usage" doesn't apply as these are team stats


In [352]:
# Create a master dictionary to hold each measure's data
#superdatadict = dict()
#for m in MeasureTypes:
#    superdatadict[m] = 0

In [438]:
#  Save Super columns for Later Work
# supercolumns = periodframe.columns.tolist()
# pickle.dump(supercolumns,open("supercolumns.p","wb"))


# Data Acquisition Script

In [19]:
# Initialize list
quarters_list = []

for D in gamedates:
        newargs.update({'DateFrom':D,'DateTo':D})
        for P in Periods:
            newargs.update({"Period":P})
            # initialize a periodframe
            periodframe = pd.DataFrame()
            for m in MeasureTypes:
                print m

                newargs.update({"MeasureType":m}) # Set MeasureType
                newline = grabdataline(payload,newargs)
                # Check for new arguments
                old_cols = periodframe.columns.tolist()
                print old_cols
                all_new_cols = newline.columns.tolist()
                newcols = list(set(all_new_cols)-set(old_cols))     
                print newcols
                if old_cols == []: # If first measure type, initialize
                    print "initializing periodframe"
                    periodframe = newline
                else:

                    # Add columns to frame
                    periodframe = periodframe.join(newline[newcols])
            
            quarters_list.append(periodframe)
            
full_frame = pd.concat(quarters_list)

Base
http://stats.nba.com/stats/leaguedashteamstats?PlusMinus=N&GameScope=&TeamID=1610612746&Location=&StarterBench=&ShotClockRange=&SeasonType=Regular+Season&Division=&Season=2014-15&PaceAdjust=N&DateFrom=11-5-2014&PlayerPosition=&VsConference=&OpponentTeamID=1610612744&DateTo=11-5-2014&GameSegment=&LastNGames=0&PlayerExperience=&VsDivision=&LeagueID=00&Outcome=&MeasureType=Base&PORound=0&PerMode=Totals&SeasonSegment=&Period=1&Rank=N&Month=0
<Response [200]>
[]
[u'W', u'MIN', u'TOV', 'Period', 'TeamID', u'REB', u'TEAM_NAME', u'W_PCT', u'FG3A', u'AST', u'PLUS_MINUS', 'DateFrom', u'FG3M', u'OREB', u'FGM', u'PF', u'PTS', u'FGA', 'OpponentTeamID', u'CFPARAMS', u'STL', 'DateTo', u'L', u'FTA', u'BLK', u'GP', u'DREB', u'FTM', u'FT_PCT', 'MeasureType', u'PFD', u'FG_PCT', u'CFID', u'TEAM_ID', u'FG3_PCT', u'BLKA']
initializing periodframe
Advanced
http://stats.nba.com/stats/leaguedashteamstats?PlusMinus=N&GameScope=&TeamID=1610612746&Location=&StarterBench=&ShotClockRange=&SeasonType=Regular+Se

In [34]:
full_frame.reset_index(inplace=True)

In [40]:
full_frame

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,OPP_FT_PCT,OPP_STL,OPP_FG3A,OPP_PTS,OPP_AST,OPP_FGA,OPP_OREB,OPP_DREB,OPP_FTM,OPP_FG3M
0,1610612746,Los Angeles Clippers,1,0,1,0,12,8,24,0.333,...,0.000,2,6,30,9,21,1,13,0,4
1,1610612746,Los Angeles Clippers,1,0,1,0,12,10,20,0.500,...,1.000,2,7,35,9,19,0,7,2,5
2,1610612746,Los Angeles Clippers,1,0,1,0,12,11,17,0.647,...,1.000,1,6,37,6,16,2,6,13,4
3,1610612746,Los Angeles Clippers,1,0,1,0,12,9,21,0.429,...,1.000,2,6,19,5,18,1,9,5,2
4,1610612746,Los Angeles Clippers,1,1,0,1,12,4,24,0.167,...,0.000,3,9,20,6,21,1,11,0,2
5,1610612746,Los Angeles Clippers,1,1,0,1,12,9,23,0.391,...,0.500,3,4,22,7,19,2,9,3,1
6,1610612746,Los Angeles Clippers,1,1,0,1,12,11,23,0.478,...,0.500,3,5,28,8,18,1,7,4,2
7,1610612746,Los Angeles Clippers,1,1,0,1,12,10,17,0.588,...,0.714,2,10,16,3,22,4,9,5,1
8,1610612746,Los Angeles Clippers,1,0,1,0,12,13,26,0.500,...,1.000,2,8,29,8,19,0,6,2,5
9,1610612746,Los Angeles Clippers,1,0,1,0,12,8,21,0.381,...,0.714,1,4,28,5,20,4,11,5,1


In [41]:
# Save into a pickle file.
pickle.dump(full_frame,open("data/fullstats_lac_gsw_20142015_103115_v3.p","wb"))
 

In [42]:
full_frame.to_csv("data/fullstats_lac_gsw_20142015_103115_v4.csv")

In [24]:
# Save full data (doesn't work???)
# full_frame.to_pickle("data/fullstats_lac_gsw_20142015_103115_v1.p")

# Functions and Constants

In [6]:
# API Call List
apivariables = ["DateFrom","DateTo","Division","GameScope","GameSegment","LastNGames","LeagueID","Location","MeasureType","Month","OpponentTeamID","Outcome","PORound","PaceAdjust","PerMode","Period","PlayerExperience","PlayerPosition","PlusMinus","Rank","Season","SeasonSegment","SeasonType","ShotClockRange","StarterBench","TeamID","VsConference","VsDivision"]
# API Initialize List
dictinit={"LastNGames":"0","LeagueID":"00","MeasureType":"Base","Month":"0","TeamID":"0","OpponentTeamID":"0","PaceAdjust":"N","PaceAdjust":"N","Period":"0","PerMode":"Totals","PlusMinus":"N","PORound":"0","Rank":"N","Season":"2014-15","SeasonType":"Regular Season"}


def init_payload():
    """Initialize payload
    
    >>> payload = init_payload()
    """
    # Initialize payload
    _payload = {}

    # Load blank payload
    _payload = _payload.fromkeys(apivariables,"")

    # Load initialized values from payloadinit
    for k,v in dictinit.iteritems():
        _payload[k] = v
    return(_payload)


def get2014gamedates(team1,team2):
    """Get gamedates in a list for two teams
    """
    # Load df
    df = pd.read_csv("data/schedule_20142015_regularseason.csv")

    # Set Date
    df['Date'] = pd.DatetimeIndex(df['Date'])

    # Get all clippers games
    df = df[df['Visitor/Neutral'].str.contains(team1) | df['Home/Neutral'].str.contains(team1)]

    # Get all warriors games
    df = df[df['Visitor/Neutral'].str.contains(team2) | df['Home/Neutral'].str.contains(team2)]
    
    # Convert to string
    
    
    # Output List
    gamedates = df['Date'].tolist()
    
    
    # Convert to string
    new_gamedates = []
    for d in gamedates:
        new_date = str(d.month) + "-" + str(d.day) + "-" + str(d.year)
        new_gamedates.append(new_date)

    gamedates = new_gamedates
    
    return(gamedates)


# Grab data
def grabdataline(payload,newargs):
    """
    Grab one line of stats for a quarter
    """
    payload.update(newargs)
    baseurl = "http://stats.nba.com/stats/leaguedashteamstats"
    response=requests.get(baseurl,payload)
    print response.url
    print response
    # Grab headers
    headers = response.json()['resultSets'][0]['headers']

    # Grab stats

    stats = response.json()['resultSets'][0]['rowSet']

    periodframe = pd.DataFrame(stats,columns = headers )
    # Add arguments to data line
    for k,v in newargs.iteritems():
        periodframe[k] = v
    return(periodframe)
def response_to_frame(requestresponse):
    """Take a request response and convert to dataframe
    Only if headers and stats conform to sample nba protocol
    
    >>> df = response_to_frame(response)
    
    """
    # Grab column heads
    headers = response.json()['resultSets'][0]['headers']

    # Grab rows
    stats = response.json()['resultSets'][0]['rowSet']
    
    # Form frame
    df = pd.DataFrame(stats,columns = headers )
    
    # Return frame
    return(df)